### Convert imported raw data into dimensional model
1. datetime format
2. data type
3. add fact measures
4. dimensional tables
5. fact table

In [165]:
import pandas as pd
from IPython.display import JSON

In [265]:
# read csv file for US videos
df = pd.read_csv("Raw Data/USvideos.csv")

In [266]:
# read json file for video category
df2 = pd.read_json('Raw Data/US_category_id.json')

In [267]:
df.head()

,video_id,trending_date,title,channel_title,category_id,publish_time,tags,views,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,video_error_or_removed,description
0,2kyS6SvSYSE,17.14.11,WE WANT TO TALK ABOUT OUR MARRIAGE,CaseyNeistat,22,2017-11-13T17:13:01.000Z,SHANtell martin,748374,57527,2966,15954,https://i.ytimg.com/vi/2kyS6SvSYSE/default.jpg,False,False,False,SHANTELL'S CHANNEL - https://www.youtube.com/s...
1,1ZAPwfrtAFY,17.14.11,The Trump Presidency: Last Week Tonight with J...,LastWeekTonight,24,2017-11-13T07:30:00.000Z,"last week tonight trump presidency|""last week ...",2418783,97185,6146,12703,https://i.ytimg.com/vi/1ZAPwfrtAFY/default.jpg,False,False,False,"One year after the presidential election, John..."
2,5qpjK5DgCt4,17.14.11,"Racist Superman | Rudy Mancuso, King Bach & Le...",Rudy Mancuso,23,2017-11-12T19:05:24.000Z,"racist superman|""rudy""|""mancuso""|""king""|""bach""...",3191434,146033,5339,8181,https://i.ytimg.com/vi/5qpjK5DgCt4/default.jpg,False,False,False,WATCH MY PREVIOUS VIDEO ▶ \n\nSUBSCRIBE ► http...
3,puqaWrEC7tY,17.14.11,Nickelback Lyrics: Real or Fake?,Good Mythical Morning,24,2017-11-13T11:00:04.000Z,"rhett and link|""gmm""|""good mythical morning""|""...",343168,10172,666,2146,https://i.ytimg.com/vi/puqaWrEC7tY/default.jpg,False,False,False,Today we find out if Link is a Nickelback amat...
4,d380meD0W0M,17.14.11,I Dare You: GOING BALD!?,nigahiga,24,2017-11-12T18:01:41.000Z,"ryan|""higa""|""higatv""|""nigahiga""|""i dare you""|""...",2095731,132235,1989,17518,https://i.ytimg.com/vi/d380meD0W0M/default.jpg,False,False,False,I know it's been a while since we did this sho...


In [268]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40949 entries, 0 to 40948
Data columns (total 16 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   video_id                40949 non-null  object
 1   trending_date           40949 non-null  object
 2   title                   40949 non-null  object
 3   channel_title           40949 non-null  object
 4   category_id             40949 non-null  int64 
 5   publish_time            40949 non-null  object
 6   tags                    40949 non-null  object
 7   views                   40949 non-null  int64 
 8   likes                   40949 non-null  int64 
 9   dislikes                40949 non-null  int64 
 10  comment_count           40949 non-null  int64 
 11  thumbnail_link          40949 non-null  object
 12  comments_disabled       40949 non-null  bool  
 13  ratings_disabled        40949 non-null  bool  
 14  video_error_or_removed  40949 non-null  bool  
 15  de

In [269]:
# object to datetime
df['trending_date'] = pd.to_datetime(df['trending_date'], format='%y.%d.%m')
df['trending_date'].head()

0   2017-11-14
1   2017-11-14
2   2017-11-14
3   2017-11-14
4   2017-11-14
Name: trending_date, dtype: datetime64[ns]

In [270]:
# object to datetime
df['publish_time'] = pd.to_datetime(df['publish_time']).dt.tz_convert(None)
df['publish_time'].head()

0   2017-11-13 17:13:01
1   2017-11-13 07:30:00
2   2017-11-12 19:05:24
3   2017-11-13 11:00:04
4   2017-11-12 18:01:41
Name: publish_time, dtype: datetime64[ns]

In [271]:
# object to string
df['video_id'] = df['video_id'].astype("string")
df['description'] = df['description'].astype("string")
df['title'] = df['title'].astype("string")
df['channel_title'] = df['channel_title'].astype("string")
df['tags'] = df['tags'].astype("string")
df['thumbnail_link'] = df['thumbnail_link'].astype("string")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40949 entries, 0 to 40948
Data columns (total 16 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   video_id                40949 non-null  string        
 1   trending_date           40949 non-null  datetime64[ns]
 2   title                   40949 non-null  string        
 3   channel_title           40949 non-null  string        
 4   category_id             40949 non-null  int64         
 5   publish_time            40949 non-null  datetime64[ns]
 6   tags                    40949 non-null  string        
 7   views                   40949 non-null  int64         
 8   likes                   40949 non-null  int64         
 9   dislikes                40949 non-null  int64         
 10  comment_count           40949 non-null  int64         
 11  thumbnail_link          40949 non-null  string        
 12  comments_disabled       40949 non-null  bool  

In [272]:
# Add fact_id as SURROGATE Key
df = df.drop_duplicates().reset_index(drop=True)
df['fact_id'] = df.index
# rename category_id
df = df.rename(columns={"category_id": "category_no"}, errors="raise")
# add column tags_count
df['tag_count'] = df['tags'].astype('string').apply(lambda x: len(x.split('|')))

df.head()

,video_id,trending_date,title,channel_title,category_no,publish_time,tags,views,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,video_error_or_removed,description,fact_id,tag_count
0,2kyS6SvSYSE,2017-11-14,WE WANT TO TALK ABOUT OUR MARRIAGE,CaseyNeistat,22,2017-11-13 17:13:01,SHANtell martin,748374,57527,2966,15954,https://i.ytimg.com/vi/2kyS6SvSYSE/default.jpg,False,False,False,SHANTELL'S CHANNEL - https://www.youtube.com/s...,0,1
1,1ZAPwfrtAFY,2017-11-14,The Trump Presidency: Last Week Tonight with J...,LastWeekTonight,24,2017-11-13 07:30:00,"last week tonight trump presidency|""last week ...",2418783,97185,6146,12703,https://i.ytimg.com/vi/1ZAPwfrtAFY/default.jpg,False,False,False,"One year after the presidential election, John...",1,4
2,5qpjK5DgCt4,2017-11-14,"Racist Superman | Rudy Mancuso, King Bach & Le...",Rudy Mancuso,23,2017-11-12 19:05:24,"racist superman|""rudy""|""mancuso""|""king""|""bach""...",3191434,146033,5339,8181,https://i.ytimg.com/vi/5qpjK5DgCt4/default.jpg,False,False,False,WATCH MY PREVIOUS VIDEO ▶ \n\nSUBSCRIBE ► http...,2,23
3,puqaWrEC7tY,2017-11-14,Nickelback Lyrics: Real or Fake?,Good Mythical Morning,24,2017-11-13 11:00:04,"rhett and link|""gmm""|""good mythical morning""|""...",343168,10172,666,2146,https://i.ytimg.com/vi/puqaWrEC7tY/default.jpg,False,False,False,Today we find out if Link is a Nickelback amat...,3,27
4,d380meD0W0M,2017-11-14,I Dare You: GOING BALD!?,nigahiga,24,2017-11-12 18:01:41,"ryan|""higa""|""higatv""|""nigahiga""|""i dare you""|""...",2095731,132235,1989,17518,https://i.ytimg.com/vi/d380meD0W0M/default.jpg,False,False,False,I know it's been a while since we did this sho...,4,14


In [273]:
# datetime_dim
datetime_dim = df[['trending_date','publish_time']].reset_index(drop=True)

datetime_dim['trending_date'] = datetime_dim['trending_date']
datetime_dim['trending_day'] = datetime_dim['trending_date'].dt.day
datetime_dim['trending_month'] = datetime_dim['trending_date'].dt.month
datetime_dim['trending_year'] = datetime_dim['trending_date'].dt.year
datetime_dim['trending_weekday'] = datetime_dim['trending_date'].dt.weekday
datetime_dim['trending_weekdayname'] = datetime_dim['trending_date'].dt.day_name().astype("string")

datetime_dim['publish_time'] = datetime_dim['publish_time']
datetime_dim['publish_hour'] = datetime_dim['publish_time'].dt.hour
datetime_dim['publish_day'] = datetime_dim['publish_time'].dt.day
datetime_dim['publish_month'] = datetime_dim['publish_time'].dt.month
datetime_dim['publish_year'] = datetime_dim['publish_time'].dt.year
datetime_dim['publish_weekday'] = datetime_dim['publish_time'].dt.weekday
datetime_dim['publish_weekdayname'] = datetime_dim['publish_time'].dt.day_name().astype("string")

datetime_dim['datetime_id'] = datetime_dim.index
# order columns 
datetime_dim = datetime_dim[['datetime_id', 'trending_date', 'trending_day', 'trending_month', 'trending_year', 'trending_weekday', 'trending_weekdayname',
                             'publish_time', 'publish_hour', 'publish_day', 'publish_month', 'publish_year', 'publish_weekday','publish_weekdayname']]
datetime_dim.head()

,datetime_id,trending_date,trending_day,trending_month,trending_year,trending_weekday,trending_weekdayname,publish_time,publish_hour,publish_day,publish_month,publish_year,publish_weekday,publish_weekdayname
0,0,2017-11-14,14,11,2017,1,Tuesday,2017-11-13 17:13:01,17,13,11,2017,0,Monday
1,1,2017-11-14,14,11,2017,1,Tuesday,2017-11-13 07:30:00,7,13,11,2017,0,Monday
2,2,2017-11-14,14,11,2017,1,Tuesday,2017-11-12 19:05:24,19,12,11,2017,6,Sunday
3,3,2017-11-14,14,11,2017,1,Tuesday,2017-11-13 11:00:04,11,13,11,2017,0,Monday
4,4,2017-11-14,14,11,2017,1,Tuesday,2017-11-12 18:01:41,18,12,11,2017,6,Sunday


In [274]:
# category_dim, match category id and category name with the json file

#JSON('Raw Data/US_category_id.json')
category_dict = {}
for item in df2['items']:
    category_dict[item['id']] = item['snippet']['title']

category_dim = df[['category_no']].reset_index(drop=True)
category_dim['category_id'] = category_dim.index
category_dim['category_title'] = category_dim['category_no'].astype('string').map(category_dict)
category_dim = category_dim[['category_id', 'category_no', 'category_title']]

category_dim.head()

,category_id,category_no,category_title
0,0,22,People & Blogs
1,1,24,Entertainment
2,2,23,Comedy
3,3,24,Entertainment
4,4,24,Entertainment


In [275]:
# title_dim

title_dim = df[['title']].reset_index(drop=True)
title_dim['title_id'] = title_dim.index
title_dim = title_dim[['title_id','title']]
title_dim.head()

,title_id,title
0,0,WE WANT TO TALK ABOUT OUR MARRIAGE
1,1,The Trump Presidency: Last Week Tonight with J...
2,2,"Racist Superman | Rudy Mancuso, King Bach & Le..."
3,3,Nickelback Lyrics: Real or Fake?
4,4,I Dare You: GOING BALD!?


In [276]:
# channel_dim

channel_dim = df[['channel_title']].reset_index(drop=True)
channel_dim['channel_id'] = channel_dim.index
channel_dim = channel_dim[['channel_id','channel_title']]
channel_dim.head()

,channel_id,channel_title
0,0,CaseyNeistat
1,1,LastWeekTonight
2,2,Rudy Mancuso
3,3,Good Mythical Morning
4,4,nigahiga


In [277]:
# tags_dim

tags_dim = df[['tags']].reset_index(drop=True)
tags_dim['tags_id'] = tags_dim.index
tags_dim = tags_dim[['tags_id','tags']]
tags_dim.head()

,tags_id,tags
0,0,SHANtell martin
1,1,"last week tonight trump presidency|""last week ..."
2,2,"racist superman|""rudy""|""mancuso""|""king""|""bach""..."
3,3,"rhett and link|""gmm""|""good mythical morning""|""..."
4,4,"ryan|""higa""|""higatv""|""nigahiga""|""i dare you""|""..."


In [278]:
# videoDesc_dim

videoDesc_dim = df[['description']].reset_index(drop=True)
videoDesc_dim['videoDesc_id'] = videoDesc_dim.index
videoDesc_dim = videoDesc_dim[['videoDesc_id','description']]
videoDesc_dim.head()

,videoDesc_id,description
0,0,SHANTELL'S CHANNEL - https://www.youtube.com/s...
1,1,"One year after the presidential election, John..."
2,2,WATCH MY PREVIOUS VIDEO ▶ \n\nSUBSCRIBE ► http...
3,3,Today we find out if Link is a Nickelback amat...
4,4,I know it's been a while since we did this sho...


In [279]:
# settings_dim

settings_dim = df[['comments_disabled', 'ratings_disabled', 'video_error_or_removed']].reset_index(drop=True)
settings_dim['settings_id'] = settings_dim.index

settings_dim = settings_dim[['settings_id','comments_disabled', 'ratings_disabled', 'video_error_or_removed']]
settings_dim.head()

,settings_id,comments_disabled,ratings_disabled,video_error_or_removed
0,0,False,False,False
1,1,False,False,False
2,2,False,False,False
3,3,False,False,False
4,4,False,False,False


In [280]:
# thumbnail_link_dim

thumbnail_link_dim = df[['thumbnail_link']].reset_index(drop=True)
thumbnail_link_dim['thumbnail_link_id'] = thumbnail_link_dim.index
thumbnail_link_dim = thumbnail_link_dim[['thumbnail_link_id','thumbnail_link']]
thumbnail_link_dim.head()

,thumbnail_link_id,thumbnail_link
0,0,https://i.ytimg.com/vi/2kyS6SvSYSE/default.jpg
1,1,https://i.ytimg.com/vi/1ZAPwfrtAFY/default.jpg
2,2,https://i.ytimg.com/vi/5qpjK5DgCt4/default.jpg
3,3,https://i.ytimg.com/vi/puqaWrEC7tY/default.jpg
4,4,https://i.ytimg.com/vi/d380meD0W0M/default.jpg


In [285]:
# fact_table

fact_table = df.merge(datetime_dim, left_on='fact_id', right_on='datetime_id') \
             .merge(category_dim, left_on='fact_id', right_on='category_id') \
             .merge(title_dim, left_on='fact_id', right_on='title_id') \
             .merge(channel_dim, left_on='fact_id', right_on='channel_id') \
             .merge(tags_dim, left_on='fact_id', right_on='tags_id') \
             .merge(videoDesc_dim, left_on='fact_id', right_on='videoDesc_id')\
             .merge(settings_dim, left_on='fact_id', right_on='settings_id') \
             .merge(thumbnail_link_dim, left_on='fact_id', right_on='thumbnail_link_id') \
             [['fact_id','video_id','datetime_id','category_id','title_id','channel_id','tags_id','videoDesc_id',
              'settings_id','thumbnail_link_id', 'tag_count', 'views', 'likes', 'dislikes', 'comment_count']]

fact_table = fact_table.rename(columns={"views": "view_count", "likes": "like_count", "dislikes": "dislike_count"})
fact_table.columns

Index(['fact_id', 'video_id', 'datetime_id', 'category_id', 'title_id',
       'channel_id', 'tags_id', 'videoDesc_id', 'settings_id',
       'thumbnail_link_id', 'tag_count', 'view_count', 'like_count',
       'dislike_count', 'comment_count'],
      dtype='object')

In [286]:
fact_table

,fact_id,video_id,datetime_id,category_id,title_id,channel_id,tags_id,videoDesc_id,settings_id,thumbnail_link_id,tag_count,view_count,like_count,dislike_count,comment_count
0,0,2kyS6SvSYSE,0,0,0,0,0,0,0,0,1,748374,57527,2966,15954
1,1,1ZAPwfrtAFY,1,1,1,1,1,1,1,1,4,2418783,97185,6146,12703
2,2,5qpjK5DgCt4,2,2,2,2,2,2,2,2,23,3191434,146033,5339,8181
3,3,puqaWrEC7tY,3,3,3,3,3,3,3,3,27,343168,10172,666,2146
4,4,d380meD0W0M,4,4,4,4,4,4,4,4,14,2095731,132235,1989,17518
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40896,40896,BZt0qjTWNhw,40896,40896,40896,40896,40896,40896,40896,40896,14,1685609,38160,1385,2657
40897,40897,1h7KV2sjUWY,40897,40897,40897,40897,40897,40897,40897,40897,1,1064798,60008,382,3936
40898,40898,D6Oy4LfoqsU,40898,40898,40898,40898,40898,40898,40898,40898,24,1066451,48068,1032,3992
40899,40899,oV0zkMe1K8s,40899,40899,40899,40899,40899,40899,40899,40899,22,5660813,192957,2846,13088
